In [ ]:
#Imports
from tcn import TCN, tcn_full_summary
import pandas as pd
import numpy  as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

In [ ]:
train_data = pd.read_csv('../Implementation/CSVFiles/2022-09-26-18-56-30-joint_states.csv')
train_data['.position'] = train_data['.position'].str.strip('()')
train_data['.velocity'] = train_data['.velocity'].str.strip('()')
train_data['.effort'] = train_data['.effort'].str.strip('()')
time_stamp = train_data[['.header.stamp.nsecs']]
position = train_data['.position'].str.split(',', expand=True)
velocity = train_data['.velocity'].str.split(',', expand=True)
effort = train_data['.effort'].str.split(',', expand=True)
features = pd.concat([position, velocity, effort], axis=1)

In [ ]:
n_features = features.shape[-1]

In [ ]:
# define the keras MLP model for x
model = Sequential()
model.add(Dense(64, input_dim=n_features, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='relu'))
# compile the keras model
model.compile(loss='mse', optimizer='adam')

In [ ]:
groundTruth = pd.read_csv("../Implementation/CSVFiles/2022-09-26-18-56-30-ground_truth-state.csv", sep =',', nrows = 3750)
Xtargets = groundTruth[['.pose.pose.position.x']]
Ytargets = groundTruth[['.pose.pose.position.y']]

In [ ]:
Xtargets = np.asarray(Xtargets).astype(np.float64)
Ytargets = np.asarray(Ytargets).astype(np.float64)

In [ ]:
features = np.asarray(features).astype(np.float64)

In [ ]:
n_features = features.shape[-1]

In [ ]:
model.fit(features, Xtargets, epochs=10000, batch_size=1024, verbose=1)

In [ ]:
yhatX = model.predict(features)
error = mean_absolute_error(Xtargets, yhatX)
print('MAE: %.3f' % error)

In [ ]:
yhatX.shape

In [ ]:
# define the keras MLP model for y
Ymodel = Sequential()
Ymodel.add(Dense(64, input_dim=n_features, activation='relu'))
Ymodel.add(Dense(32, activation='relu'))
Ymodel.add(Dense(32, activation='relu'))
Ymodel.add(Dense(1, activation='relu'))
# compile the keras model
Ymodel.compile(loss='mse', optimizer='adam')

In [ ]:
Ymodel.fit(features, Ytargets, epochs=10000, batch_size=1024, verbose=1)

In [ ]:
yhatY = Ymodel.predict(features)
error = mean_absolute_error(Ytargets, yhatY)
print('MAE: %.3f' % error)

In [ ]:
plt.scatter(yhatX, yhatY, color='red')
plt.scatter(Xtargets, Ytargets, color='blue')
plt.show()